In [1]:
import wrds
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
from scipy import stats
import pickle
from pandas.tseries.offsets import BDay
from pandas.tseries.offsets import DateOffset

In [3]:
db = wrds.Connection(wrds_username='novenwz')
#db.create_pgpass_file()

WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [2]:
# Get options data and security prices
def GetData(stock):
    # Find the security id
    sql = '''
    SELECT secid
    FROM optionm.secnmd
    WHERE ticker='%s'
    ''' % stock
    secid = db.raw_sql(sql).loc[:,'secid']
    secid = np.unique(secid)
    
    df = pd.DataFrame()
    # Keep track of the positino of secid
    pos = 0
    while df.empty and pos<=len(secid):
        # Get the data for the options
        sql = '''
        SELECT secid, date, exdate, 
        strike_price, impl_volatility,open_interest,cp_flag,
        forward_price,best_bid,best_offer
        FROM optionm.opprcd2009
        WHERE secid=%s
        AND date >= '1996-06-30'
        ORDER BY date ASC
        ''' % (secid[pos])

        df_2009 = db.raw_sql(sql)

        years = range(1996,2019)

        df10to19 = pd.DataFrame()
        for year in years:
            sql = '''
            SELECT secid, date, exdate,
            strike_price, impl_volatility,open_interest,cp_flag,
            forward_price,best_bid,best_offer
            FROM optionm.opprcd%d
            WHERE secid=%s
            ORDER BY date ASC
            ''' % (year,secid[pos])

            df10to19 = pd.concat([df10to19, db.raw_sql(sql)],axis=0)

        sql = '''
        SELECT secid, date, exdate, 
        strike_price, impl_volatility,open_interest,cp_flag,
        forward_price,best_bid,best_offer
        FROM optionm.opprcd2019
        WHERE secid=%s
        AND date <= '2019-06-30'

        ORDER BY date ASC
        ''' % (secid[pos])

        df_2019 = db.raw_sql(sql)

        df = pd.concat([df_2009,df10to19,df_2019],axis=0)
        df = df.reset_index().drop(columns='index')

        # Get the security prices
        sql = '''
        SELECT date, close
        FROM optionm.secprd
        WHERE secid=%s
        AND date BETWEEN '1996-06-30' AND '2019-06-30'
        ORDER BY date ASC
        ''' % secid[pos]

        price = db.raw_sql(sql)
        close = price['close']
        close.index = price['date']
        pos+=1

    return df, close

In [15]:
sp = GetData("SPX")

In [16]:
sp[0]

,secid,date,exdate,strike_price,impl_volatility,open_interest,cp_flag,forward_price,best_bid,best_offer
0,109820.0,2009-06-30,2010-12-31,60000.0,0.372396,538.0,P,93.630113,2.83,2.99
1,109820.0,2009-06-30,2010-12-31,61000.0,0.370599,42.0,P,93.630113,3.00,3.20
2,109820.0,2009-06-30,2010-12-31,62000.0,0.366784,44.0,P,93.630113,3.15,3.35
3,109820.0,2009-06-30,2010-12-31,63000.0,0.364829,42.0,P,93.630113,3.35,3.55
4,109820.0,2009-06-30,2010-12-31,64000.0,0.362576,74.0,P,93.630113,3.55,3.75
5,109820.0,2009-06-30,2010-12-31,50000.0,0.398539,646.0,P,93.630113,1.45,1.60
6,109820.0,2009-06-30,2010-12-18,95000.0,0.225161,28058.0,C,93.551318,9.20,9.50
7,109820.0,2009-06-30,2010-12-18,70000.0,0.238143,1654.0,C,93.551318,24.40,25.65
8,109820.0,2009-06-30,2010-12-18,45000.0,NaN,216.0,C,93.551318,46.50,47.95
9,109820.0,2009-06-30,2010-12-18,20000.0,NaN,1.0,C,93.551318,70.40,73.60
